In order to execute TravOrder there must be a list of the cities as well as a dictionary of all the distances between the cities created

In [3]:
# First we must represent a list of cities that will be used in graph representation
cities = ['B', 'L', 'M', 'S']

# create undirected graph representation
dist = {
    ('B', 'L'): 3.0,
    ('L', 'B'): 3.0,
    ('L', 'M'): 4.5,
    ('M', 'L'): 4.5,
    ('M', 'S'): 3.1,
    ('S', 'M'): 3.1,
    ('S', 'L'): 5.7,
    ('L', 'S'): 5.7,
    ('B', 'M'): 7.6,
    ('M', 'B'): 7.6,
    ('B', 'S'): 7.8,
    ('S', 'B'): 7.8,
}   

# create a helper function to get the distance of each tuple
def distance(c1, c2):
    return dist[(c1, c2)]

Now that we have a list of the cities and the dictionaries, as well as a way to access the dictionary easily we need to sort the neighbors to be able to get the k-th closest unvisited city for the algorithm

In [4]:
from collections import defaultdict

# initialize defaultdict to store adjacency list representation of graph
neighbors = defaultdict(list)
# iterate through distances in dist dictionary
for (c1, c2), d in dist.items():
    neighbors[c1].append((c2, d))

# sort each neighbor list by distance
for city in neighbors:
    neighbors[city].sort(key=lambda x: x[1]) # sorting is based on distance from shortest to longest

In [ ]:
def trav_order(start, cities_list, neighbors_map, gene):
    # instantiate the unvisited cities list and remove the start city
    unvisited = set(cities_list)
    unvisited.remove(start)

    route = [start]
    total_distance = 0.0
    current_city = start
    # need to visit each city exactly once
    steps_needed = len(cities_list) - 1
    # loop through every city needed to visit
    for step in range(steps_needed):
        if step < len(gene):  # if the step is less than the length, then add the rank
            rank = gene[step]
        else:
            rank = 1 # default to closest unvisited city
        candidate_neighbors = [
            (nbr, d) for nbr, d in neighbors_map[current_city] if nbr in unvisited
        ]
    
        # wrap the rank if it exceeds the number of available neighbors
        idx = (rank - 1) % len(candidate_neighbors)
        next_city, step_distance = candidate_neighbors[idx]

        # update route and tracking variables
        route.append(next_city)
        total_distance += step_distance
        unvisited.remove(next_city)
        current_city = next_city
    # update distance variables for the complete route
    return_distance = distance(current_city, start)
    total_distance += return_distance
    # close route by returning to start city
    route.append(start)
    
    return route, total_distance


Now prove the function works:

In [6]:
gene = [1, 1, 1]
test_route, total_distance = trav_order('B', cities, neighbors, gene)
print("Route Taken:", test_route)
print("Total Distance:", total_distance)

gene = [2, 1, 1]
test_route, total_distance = trav_order('L', cities, neighbors, gene)
print("Route Taken:", test_route)
print("Total Distance:", total_distance)

Route Taken: ['B', 'L', 'M', 'S', 'B']
Total Distance: 18.4
Route Taken: ['L', 'M', 'S', 'B', 'L']
Total Distance: 18.4


Now implement crossover function that implements a one point cut and crossover. This function does not need to know about cities or distances, simply recombines two integer sequences

In [7]:
import random
def crossover(parent1, parent2):
    # ensure the parent lengths are the same for crossover
    if len(parent1) != len(parent2):
        raise ValueError("Parents must have the same length for crossover")
    
    # get the length of the chromosome
    n = len(parent1)
    # randomly choose the cut point for the crossover between 1 and n - 1
    # to avoid 0 and n cases where there is no crossover and 100% of genes from one parent are chosen
    cut_point = random.randint(1, n - 1)

    # complete the cut and crossover in one step with simple list splicing
    child = parent1[:cut_point] + parent2[cut_point:]
    return child

Create mutate function with a mutate probability. This will mutate a maximum of 1 position per child

In [8]:
def mutate(gene, mutation_prob=0.2):
    rng = random
    # decide if this child will have a mutation
    if rng.random() > mutation_prob:
        return gene[:] # no mutation occurs, return copy of original gene
    
    n = len(gene)
    
    max_rank = n # bound the max rank a mutation can have
    new_gene = gene[:] # make a copy of the existing gene

    # choose the position to mutate
    position = rng.randrange(n)
    # get the old rank in the original position that will be mutated
    old_rank = new_gene[position]
    new_rank = rng.randint(1, max_rank)
    # if the new rank is the same, then generate another rank
    while new_rank == old_rank: 
        new_rank = rng.randint(1, max_rank)
    new_gene[position] = new_rank # set the new rank
    return new_gene

Now implement the genetic algorithm
Requires function to generate random genes, evaluate the genes, and select the best genes from population

In [ ]:
def random_gene(num_steps, max_rank=None):
    # Create a random rank-encoded gene.
    if max_rank is None:
        max_rank = num_steps  # 
    return [random.randint(1, max_rank) for _ in range(num_steps)]


# evaluate whether a gene is good and return the route, total distance, and fitness
def evaluate_gene(gene, start_city, cities_list, neighbors_map):
    # complete route and fitness for gene
    route, total_distance = trav_order(start_city, cities_list, neighbors_map, gene)
    fitness = 1.0/total_distance
    return route, total_distance, fitness


def select_best_gene(population, fitness, k=3):
    # select k random genes and return the best
    indices = random.sample(range(len(population)), k=k)
    best_idx = max(indices, key=lambda i: fitness[i])
    return population[best_idx]

Now create the main genetic algorithm loop to run the code

In [10]:
def run_ga(
        start_city,
        cities_list,
        neighbors_map,
        population_size=20,
        num_generations=50,
        mutation_prob=0.2,
        best_k=3
):
    num_steps = len(cities_list) - 1
    # generate the initial population
    population = [random_gene(num_steps) for _ in range(population_size)]

    # instantiate variables we are looking for
    best_gene = None
    best_route = None
    best_distance = float("inf")

    for gen in range(num_generations):
        # evaluate current population
        fitnesses = []
        for gene in population:
            route, total_dist, fitness = evaluate_gene(gene, start_city, cities_list, neighbors_map)
            fitnesses.append(fitness)
            # if the current total distance is better than best, update best values
            if total_dist < best_distance:
                best_distance = total_dist
                best_gene = gene[:]
                best_route = route[:]
        # create next generation values using crossover and mutation
        new_population = []
        while len(new_population) < population_size:
            # select parents
            parent1 = select_best_gene(population, fitnesses, k=best_k)
            parent2 = select_best_gene(population, fitnesses, k=best_k)

            # complete crossover
            child_gene = crossover(parent1, parent2)

            # mutate child gene
            child_gene = mutate(child_gene, mutation_prob=mutation_prob)

            new_population.append(child_gene)
            
        population = new_population
            
    return best_gene, best_route, best_distance

Now run the genetic algorithm with trav_order on the existing dataset 

In [11]:
best_gene, best_route, best_distance = run_ga(start_city='B', 
                                              cities_list=cities, 
                                              neighbors_map=neighbors, 
                                              population_size=30,
                                              num_generations=50,
                                              mutation_prob=0.2,
                                              best_k=3)

print("Best gene: ", best_gene)
print("Best route: ", best_route)
print("Best distance: ", best_distance)

Best gene:  [1, 3, 2]
Best route:  ['B', 'L', 'M', 'S', 'B']
Best distance:  18.4


Add more cities to create a fully connected, undirected graph

In [12]:
# 20 city labels
cities = [
    'A', 'B', 'C', 'D', 'E',
    'F', 'G', 'H', 'I', 'J',
    'K', 'L', 'M', 'N', 'O',
    'P', 'Q', 'R', 'S', 'T'
]

# create an undirected, fully-connected distance dictionary
dist = {}

used_distances = set()

for i in range(len(cities)):
    for j in range(i + 1, len(cities)):
        c1 = cities[i]
        c2 = cities[j]

        # generate a unique distance for the city pair
        while True:
            d = round(random.uniform(1.0, 100.0), 3) # generate a new potential distance
            if d not in used_distances: # ensure the newly generated distance is not used
                used_distances.add(d)
                break

        # undirected graph → store both directions
        dist[(c1, c2)] = d
        dist[(c2, c1)] = d


# rebuild neighbours
# rebuild neighbors for the new 20-city dist
neighbors = defaultdict(list)
for (c1, c2), d in dist.items():
    neighbors[c1].append((c2, d))

# sort each neighbor list by distance (shortest first)
for city in neighbors:
    neighbors[city].sort(key=lambda x: x[1])


In [13]:
best_gene, best_route, best_distance = run_ga(start_city='L', 
                                              cities_list=cities, 
                                              neighbors_map=neighbors, 
                                              population_size=30,
                                              num_generations=50,
                                              mutation_prob=0.2,
                                              best_k=3)

print("Best gene: ", best_gene)
print("Best route: ", best_route)
print("Best distance: ", best_distance)

greedy_gene = [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
greedy_route, greedy_distance = trav_order('L', cities, neighbors, greedy_gene)
print("Greedy route: ", greedy_route)
print("Greedy distance: ", greedy_distance)

Best gene:  [2, 3, 4, 17, 17, 18, 12, 15, 18, 12, 1, 2, 1, 1, 7, 1, 7, 5, 3]
Best route:  ['L', 'C', 'Q', 'R', 'I', 'H', 'A', 'D', 'F', 'P', 'S', 'T', 'E', 'N', 'B', 'O', 'K', 'J', 'G', 'M', 'L']
Best distance:  477.51500000000004
Greedy route:  ['L', 'B', 'K', 'J', 'G', 'A', 'N', 'E', 'D', 'O', 'Q', 'R', 'C', 'P', 'S', 'T', 'F', 'H', 'I', 'M', 'L']
Greedy distance:  319.8450000000001


Increase to 30 cities

In [14]:
# 30 city labels
cities = [
    'A', 'B', 'C', 'D', 'E',
    'F', 'G', 'H', 'I', 'J',
    'K', 'L', 'M', 'N', 'O',
    'P', 'Q', 'R', 'S', 'T',
    'U', 'V', 'W', 'X', 'Y',
    'Z', '1', '2', '3', '4'
]

# create an undirected, fully-connected distance dictionary
dist = {}

used_distances = set()

for i in range(len(cities)):
    for j in range(i + 1, len(cities)):
        c1 = cities[i]
        c2 = cities[j]

        # generate a unique distance for the city pair
        while True:
            d = round(random.uniform(1.0, 100.0), 3) # generate a new potential distance
            if d not in used_distances: # ensure the newly generated distance is not used
                used_distances.add(d)
                break

        # undirected graph → store both directions
        dist[(c1, c2)] = d
        dist[(c2, c1)] = d


# rebuild neighbours
# rebuild neighbors for the new 20-city dist
neighbors = defaultdict(list)
for (c1, c2), d in dist.items():
    neighbors[c1].append((c2, d))

# sort each neighbor list by distance (shortest first)
for city in neighbors:
    neighbors[city].sort(key=lambda x: x[1])


In [15]:
best_gene, best_route, best_distance = run_ga(start_city='T', 
                                              cities_list=cities, 
                                              neighbors_map=neighbors, 
                                              population_size=200,
                                              num_generations=300,
                                              mutation_prob=0.5,
                                              best_k=3)

print("Best gene: ", best_gene)
print("Best route: ", best_route)
print("Best distance: ", best_distance)


Best gene:  [8, 2, 28, 27, 3, 26, 25, 1, 1, 23, 20, 19, 18, 1, 16, 1, 17, 2, 1, 1, 29, 17, 22, 7, 26, 13, 22, 17, 13]
Best route:  ['T', 'Y', 'Z', '4', 'E', 'W', '3', 'S', 'J', 'G', '1', 'D', 'M', 'C', 'Q', 'H', 'X', 'A', 'R', 'F', 'O', 'L', 'B', 'I', 'V', 'N', 'K', 'P', 'U', '2', 'T']
Best distance:  315.7849999999999


In [16]:
greedy_gene = [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
greedy_route, greedy_distance = trav_order('T', cities, neighbors, greedy_gene)
print("Greedy route: ", greedy_route)
print("Greedy distance: ", greedy_distance)

Greedy route:  ['T', '4', 'Z', 'K', 'S', 'J', 'G', 'W', 'A', 'N', 'V', 'D', 'M', 'C', 'Q', '3', 'O', 'P', 'U', 'B', 'X', 'I', 'F', 'R', 'E', 'L', '2', '1', 'Y', 'H', 'T']
Greedy distance:  444.329


Now we will enhance the algorithm by making it run multiple times, and do a parameter search for the best trav_order parameters.

In [39]:
import itertools
import statistics


def parameter_search(start_city, cities_list, neighbors_map, population_sizes,
                     num_generations_list, mutation_probs, best_k_list, 
                     trials_per_setting=5):
    all_results = []
    best_overall = None
    # Cartesian product of all hyperparameter choices
    for pop_size, n_gen, mut_prob, k in itertools.product(population_sizes,
                                                          num_generations_list,
                                                          mutation_probs,
                                                          best_k_list):
        trial_distances = []
        trial_genes = []
        trial_routes = []

        for t in range(trials_per_setting):
            random.seed(89)
            best_gene, best_route, best_distance = run_ga(
                start_city=start_city,
                cities_list=cities_list,
                neighbors_map=neighbors_map,
                population_size=pop_size,
                num_generations=n_gen,
                mutation_prob=mut_prob,
                best_k=k
            )

            trial_distances.append(best_distance)
            trial_genes.append(best_gene)
            trial_routes.append(best_route)
        
        avg_distance=statistics.mean(trial_distances)
        best_idx=min(range(len(trial_distances)), key=lambda i: trial_distances[i])
        result = {
            "population_size": pop_size,
            "num_generations": n_gen,
            "mutation_prob": mut_prob,
            "best_k": k,
            "avg_distance": avg_distance,
            "best_distance": trial_distances[best_idx],
            "best_gene": trial_genes[best_idx],
            "best_route": trial_routes[best_idx],
        }
        all_results.append(result)

        if best_overall is None or avg_distance < best_overall['avg_distance']:
            best_overall = result

    return best_overall, all_results



Run parameter search example

In [40]:
population_sizes = [50, 100, 200]
num_generations_list = [100, 200, 300]
mutation_probs = [0.2, 0.4, 0.6]
best_k_list = [2, 3, 5]

best_overall, all_results = parameter_search(
    start_city='T',
    cities_list=cities,
    neighbors_map=neighbors,
    population_sizes=population_sizes,
    num_generations_list=num_generations_list,
    mutation_probs=mutation_probs,
    best_k_list=best_k_list,
    trials_per_setting=5,   # adjust up/down as you like
)

print("\n=== Best overall setting ===")
print("population_size:", best_overall["population_size"])
print("num_generations:", best_overall["num_generations"])
print("mutation_prob :", best_overall["mutation_prob"])
print("best_k        :", best_overall["best_k"])
print("best_distance :", best_overall["best_distance"])
print("best_gene     :", best_overall["best_gene"])
print("best_route    :", best_overall["best_route"])


=== Best overall setting ===
population_size: 200
num_generations: 100
mutation_prob : 0.2
best_k        : 2
best_distance : 270.803
best_gene     : [5, 6, 2, 3, 28, 6, 24, 24, 1, 21, 2, 22, 18, 1, 3, 1, 1, 26, 1, 2, 2, 9, 22, 13, 21, 17, 7, 23, 27]
best_route    : ['T', '2', '1', 'V', 'I', 'B', 'F', 'R', 'S', 'J', 'G', '3', 'H', 'C', 'Q', 'M', 'D', 'X', 'Y', 'Z', 'K', 'N', 'A', 'W', 'U', 'P', 'O', 'L', 'E', '4', 'T']


We can see that after implementing a very simple parameter search the best route becomes far shorter than the greedy method and assists in choosing the best parameters. We will now implement a very simple permutation based genetic algorithm to the traveling salesman problem

In [34]:
# add functions for permutation GA traveling salesman implementation

def decode_route_perm(start_city, perm):
    # get the route given a start city and permutation
    route = [start_city] + perm + [start_city]
    total_distance = 0.0
    # iterate through route and add up total distance of route
    for i in range(len(route) - 1):
        total_distance += distance(route[i], route[i+1])
    return route, total_distance


def random_perm_gene(cities_list, start_city):
    # return a random permutation containing all cities except start city
    perm = [c for c in cities_list if c != start_city]
    random.shuffle(perm)
    return perm


def evaluate_perm_gene(gene, start_city):
    # get the route, distance, and fitness for a gene thats perm-encoded
    route, total_distance = decode_route_perm(start_city, gene)
    fitness = 1.0 / total_distance
    return route, total_distance, fitness

# reuse the best gene code - no changes necessary
def select_best_gene(population, fitness, k=3):
    # select k random genes and return the best
    indices = random.sample(range(len(population)), k=k)
    best_idx = max(indices, key=lambda i: fitness[i])
    return population[best_idx]

Crossover function now maintains order of permutation, copies slice from parent 1, populates remainder of chromosome with parent 2

In [19]:
def order_crossover(parent1, parent2):
    # get the length of parent 1
    n = len(parent1)

    # choose a random slice [i, j) - this will be copied to child
    i = random.randint(0, n - 2)
    j = random.randint(i + 1, n - 1)

    # create a child chromosome, populate with None values same length as parent 1
    child = [None] * n

    # copy slice from parent1
    child[i:j] = parent1[i:j]

    # fill remaining positions from parent2 in order,
    p2_idx = 0
    for k in range(n):
        if child[k] is None:
            # find next city in parent2 that is not yet in child
            while parent2[p2_idx] in child:
                # move to next index if city already in child
                p2_idx += 1
            # if value is not in child, then populate and move to next index    
            child[k] = parent2[p2_idx]
            p2_idx += 1

    return child


Mutation function uses swap method with probability, selects two random indicies and swaps the values

In [20]:
def mutate_swap(gene, mutation_prob=0.2):
    # decide if mutation will occur
    if random.random() > mutation_prob:
        return gene[:]

    # get the length of the gene
    n = len(gene)
    # choose two random indicies in range of length of gene
    i, j = random.sample(range(n), 2)
    new_gene = gene[:]
    # swap the values of the positions and return the new gene
    new_gene[i], new_gene[j] = new_gene[j], new_gene[i]
    return new_gene


In [29]:
def run_ga_perm(
    start_city,
    cities_list,
    population_size=100,
    num_generations=200,
    mutation_prob=0.6,
    best_k=3,
):
    # initialize population
    population = [
        random_perm_gene(cities_list, start_city)
        for _ in range(population_size)
    ]

    best_gene = None
    best_route = None
    best_distance = float("inf")

    for gen in range(num_generations):
        fitnesses = []
        for gene in population:
            route, total_dist, fitness = evaluate_perm_gene(gene, start_city)
            fitnesses.append(fitness)

            if total_dist < best_distance:
                best_distance = total_dist
                best_gene = gene[:]
                best_route = route[:]

        # build next generation
        new_population = []
        while len(new_population) < population_size:
            parent1 = select_best_gene(population, fitnesses, k=best_k)
            parent2 = select_best_gene(population, fitnesses, k=best_k)

            child = order_crossover(parent1, parent2)
            child = mutate_swap(child, mutation_prob=mutation_prob)
            new_population.append(child)

        population = new_population

    return best_gene, best_route, best_distance


In [30]:
# New permutation-based GA
best_gene_perm, best_route_perm, best_dist_perm = run_ga_perm(
    start_city='T',
    cities_list=cities,
    population_size=200,
    num_generations=300,
    mutation_prob=0.2,
    best_k=3,
)

print("Perm GA best distance:", best_dist_perm)
print("Perm GA best route   :", best_route_perm)

Perm GA best distance: 329.05100000000004
Perm GA best route   : ['T', 'N', 'S', 'J', 'K', 'Z', 'Y', 'B', 'I', 'V', '1', 'G', '3', 'O', 'P', 'U', 'W', 'A', 'X', 'F', 'R', 'L', '2', 'D', 'M', 'C', 'H', 'Q', 'E', '4', 'T']


Now we will implement the same strategy of searching for the best parameters for the permutation approach in order to compare the results from the two algorithms

In [ ]:
def parameter_search(start_city, cities_list, neighbors_map, population_sizes,
                     num_generations_list, mutation_probs, best_k_list, 
                     trials_per_setting=5, perm=False):
    all_results = []
    best_overall = None
    # Cartesian product of all hyperparameter choices
    for pop_size, n_gen, mut_prob, k in itertools.product(population_sizes,
                                                          num_generations_list,
                                                          mutation_probs,
                                                          best_k_list):
        trial_distances = []
        trial_genes = []
        trial_routes = []

        for t in range(trials_per_setting):
            random.seed(89)
            # if perm flag is set, then run perm GA implementation
            if perm is True:
                best_gene, best_route, best_distance = run_ga_perm(
                    start_city=start_city,
                    cities_list=cities_list,
                    population_size=pop_size,
                    num_generations=n_gen,
                    mutation_prob=mut_prob,
                    best_k=k
                )
            # else run with rank GA implementation
            else: 
                best_gene, best_route, best_distance = run_ga(
                    start_city=start_city,
                    cities_list=cities_list,
                    neighbors_map=neighbors_map,
                    population_size=pop_size,
                    num_generations=n_gen,
                    mutation_prob=mut_prob,
                    best_k=k
                )

            trial_distances.append(best_distance)
            trial_genes.append(best_gene)
            trial_routes.append(best_route)
        
        avg_distance=statistics.mean(trial_distances)
        best_idx=min(range(len(trial_distances)), key=lambda i: trial_distances[i])
        result = {
            "population_size": pop_size,
            "num_generations": n_gen,
            "mutation_prob": mut_prob,
            "best_k": k,
            "avg_distance": avg_distance,
            "best_distance": trial_distances[best_idx],
            "best_gene": trial_genes[best_idx],
            "best_route": trial_routes[best_idx],
        }
        all_results.append(result)

        if best_overall is None or avg_distance < best_overall['avg_distance']:
            best_overall = result

    return best_overall, all_results



In [ ]:
# run multiple parameter search for the permutation implementation
population_sizes = [50, 100, 200]
num_generations_list = [100, 200, 300]
mutation_probs = [0.4, 0.6, 0.8]
best_k_list = [2, 3, 5]

best_overall, all_results = parameter_search(
    start_city='T',
    cities_list=cities,
    neighbors_map=neighbors,
    population_sizes=population_sizes,
    num_generations_list=num_generations_list,
    mutation_probs=mutation_probs,
    best_k_list=best_k_list,
    trials_per_setting=5,   # adjust up/down as you like
    perm=True  # set perm flag to be true for this iteration to test
)

print("\n=== Best overall setting for permutation implementation ===")
print("population_size:", best_overall["population_size"])
print("num_generations:", best_overall["num_generations"])
print("mutation_prob :", best_overall["mutation_prob"])
print("best_k        :", best_overall["best_k"])
print("best_distance :", best_overall["best_distance"])
print("best_gene     :", best_overall["best_gene"])
print("best_route    :", best_overall["best_route"])


=== Best overall setting for permutation implementation ===
population_size: 200
num_generations: 200
mutation_prob : 0.8
best_k        : 5
avg_distance  : 312.8769999999999
best_distance : 312.8769999999999
best_gene     : ['Y', 'Z', 'W', '3', 'Q', 'H', 'C', 'M', 'D', 'S', 'K', 'N', 'V', '1', 'G', 'J', 'A', '2', 'L', 'O', 'P', 'U', 'B', 'X', 'I', 'F', 'R', 'E', '4']
best_route    : ['T', 'Y', 'Z', 'W', '3', 'Q', 'H', 'C', 'M', 'D', 'S', 'K', 'N', 'V', '1', 'G', 'J', 'A', '2', 'L', 'O', 'P', 'U', 'B', 'X', 'I', 'F', 'R', 'E', '4', 'T']


Now compare results with rank GA implementation

In [ ]:
# run multiple parameter search for the rank GA implementation
population_sizes = [50, 100, 200]
num_generations_list = [100, 200, 300]
mutation_probs = [0.4, 0.6, 0.8]
best_k_list = [2, 3, 5]

best_overall, all_results = parameter_search(
    start_city='T',
    cities_list=cities,
    neighbors_map=neighbors,
    population_sizes=population_sizes,
    num_generations_list=num_generations_list,
    mutation_probs=mutation_probs,
    best_k_list=best_k_list,
    trials_per_setting=5,   # adjust up/down as you like
)

print("\n=== Best overall setting for rank implementation ===")
print("population_size:", best_overall["population_size"])
print("num_generations:", best_overall["num_generations"])
print("mutation_prob :", best_overall["mutation_prob"])
print("best_k        :", best_overall["best_k"])
print("best_distance :", best_overall["best_distance"])
print("best_gene     :", best_overall["best_gene"])
print("best_route    :", best_overall["best_route"])


=== Best overall setting for rank implementation ===
population_size: 100
num_generations: 200
mutation_prob : 0.8
best_k        : 2
avg_distance  : 290.09899999999993
best_distance : 290.09899999999993
best_gene     : [8, 6, 29, 2, 27, 26, 4, 23, 22, 1, 20, 19, 19, 18, 17, 1, 3, 26, 1, 2, 10, 25, 8, 19, 16, 25, 7, 15, 12]
best_route    : ['T', 'Y', 'M', 'C', 'H', 'Q', 'S', 'K', 'Z', '4', 'E', 'R', 'F', 'I', 'V', 'N', 'A', 'P', 'U', 'W', 'G', 'J', 'B', 'X', 'D', '1', '3', 'O', 'L', '2', 'T']


Attempt to improve trav_order by avoiding building candidates list in each loop

In [43]:
MAX_RANK = 5 

def random_gene(num_steps, max_rank=MAX_RANK):
    # Create a random rank-encoded gene with ranks in [1, max_rank]
    return [random.randint(1, max_rank) for _ in range(num_steps)]


def mutate(gene, mutation_prob=0.2, max_rank=MAX_RANK):
    rng = random
    if rng.random() > mutation_prob:
        return gene[:]  # no mutation, just copy

    n = len(gene)
    new_gene = gene[:]

    position = rng.randrange(n)
    old_rank = new_gene[position]
    new_rank = rng.randint(1, max_rank)
    while new_rank == old_rank and max_rank > 1:
        new_rank = rng.randint(1, max_rank)
    new_gene[position] = new_rank

    return new_gene

def trav_order(start, cities_list, neighbors_map, gene):
    unvisited = set(cities_list)
    unvisited.remove(start)

    route = [start]
    total_distance = 0.0
    current_city = start
    steps_needed = len(cities_list) - 1

    for step in range(steps_needed):
        # get rank for this step, default to 1 if gene shorter
        if step < len(gene):
            rank = gene[step]
        else:
            rank = 1

        # collect up to MAX_RANK nearest unvisited neighbors
        candidates = []
        for nbr, d in neighbors_map[current_city]:
            if nbr in unvisited:
                candidates.append((nbr, d))
                if len(candidates) == MAX_RANK:
                    break

        if not candidates:
            raise RuntimeError(
                f"No unvisited neighbors left from {current_city}"
            )

        # wrap rank into the available candidates
        idx = (rank - 1) % len(candidates)
        next_city, step_distance = candidates[idx]

        route.append(next_city)
        total_distance += step_distance
        unvisited.remove(next_city)
        current_city = next_city

    # close the route
    return_distance = distance(current_city, start)
    total_distance += return_distance
    route.append(start)

    return route, total_distance


In [44]:
# run multiple parameter search for the rank GA implementation
population_sizes = [50, 100, 200]
num_generations_list = [100, 200, 300]
mutation_probs = [0.4, 0.6, 0.8]
best_k_list = [2, 3, 5]

best_overall, all_results = parameter_search(
    start_city='T',
    cities_list=cities,
    neighbors_map=neighbors,
    population_sizes=population_sizes,
    num_generations_list=num_generations_list,
    mutation_probs=mutation_probs,
    best_k_list=best_k_list,
    trials_per_setting=5,   # adjust up/down as you like
)

print("\n=== Best overall setting for rank implementation ===")
print("population_size:", best_overall["population_size"])
print("num_generations:", best_overall["num_generations"])
print("mutation_prob :", best_overall["mutation_prob"])
print("best_k        :", best_overall["best_k"])
print("best_distance :", best_overall["best_distance"])
print("best_gene     :", best_overall["best_gene"])
print("best_route    :", best_overall["best_route"])


=== Best overall setting for rank implementation ===
population_size: 200
num_generations: 300
mutation_prob : 0.6
best_k        : 2
best_distance : 236.12899999999996
best_gene     : [1, 2, 1, 1, 2, 2, 2, 1, 1, 1, 3, 1, 1, 1, 1, 1, 4, 1, 1, 1, 1, 2, 1, 1, 1, 1, 4, 1, 4]
best_route    : ['T', '4', 'E', 'R', 'F', 'I', 'V', 'N', 'A', 'W', '3', 'G', 'J', 'S', 'K', 'Z', 'B', 'U', 'P', 'O', 'L', '2', '1', 'D', 'M', 'C', 'Q', 'H', 'X', 'Y', 'T']
